In [13]:
from dotenv import load_dotenv
import os
import google.generativeai as genai

# load and set API key
load_dotenv()
gemini_api_key = os.getenv("GEMINI_API_KEY")
# set up Gemini
genai.configure(api_key=gemini_api_key)
model = genai.GenerativeModel('gemini-1.5-flash')

# simple prompt
def get_response(prompt):
    generation_config = genai.types.GenerationConfig(temperature=0.8)
    response = model.generate_content(prompt, generation_config=generation_config)
    return response

### Create employee data

In [14]:
import json

prompt1 = [
    {"parts": [{"text": """create fake employee data for a big Belgian telecom company (Proximus), and return in a json format. 
                It should contain 20 employees, including for each name: function, department, e-mail, and preferred language (Dutch, French or English). 
                Have no more than 5 departments, and make sure all of the company activieties fall into those departments. 
                Do not return anything other than the json."""}]}
]
result = get_response(prompt1)

# Extract the answer
clean_result = result.text[7:].strip()[:-3].strip()

# create employee list, departments dict
employee_data = json.loads(clean_result)
print("Employees: ", employee_data)

departments = {employee['department'] for employee in employee_data}
print("Departments: ", departments)
## add data to include a description to departments (Tavily/Google search for Proximus company structure)

Employees:  [{'name': 'Jan De Vries', 'function': 'Software Engineer', 'department': 'Technology', 'email': 'jan.devries@proximus.be', 'preferred_language': 'Dutch'}, {'name': 'Marie Dubois', 'function': 'Network Engineer', 'department': 'Technology', 'email': 'marie.dubois@proximus.be', 'preferred_language': 'French'}, {'name': 'Peter Jones', 'function': 'Project Manager', 'department': 'Product Development', 'email': 'peter.jones@proximus.be', 'preferred_language': 'English'}, {'name': 'Annelies Janssen', 'function': 'Data Analyst', 'department': 'Business Intelligence', 'email': 'annelies.janssen@proximus.be', 'preferred_language': 'Dutch'}, {'name': 'Jean-Pierre Lefevre', 'function': 'Sales Representative', 'department': 'Sales & Marketing', 'email': 'jean-pierre.lefevre@proximus.be', 'preferred_language': 'French'}, {'name': 'Sarah Smith', 'function': 'Marketing Manager', 'department': 'Sales & Marketing', 'email': 'sarah.smith@proximus.be', 'preferred_language': 'English'}, {'nam

### Create list of senders

In [15]:
# Microsoft products

# Local companies

### Get company logos

In [32]:
from tavily import TavilyClient
import requests

company_name = "Proximus"

def get_logo(company_name):
    """Takes company name as an input, saves company logo to file"""
    # Instantiating  TavilyClient
    load_dotenv()
    tavily_api_key = os.getenv("TAVILY_API_KEY")
    client = TavilyClient(api_key=tavily_api_key)
    # Logo search (returns 5 logo urls, with description)
    response = client.search(f"{company_name} logo", include_images=True, include_image_descriptions=True)
    # Gemini picks a favourite
    prompt3 = [
        {"parts": [{"text": f"""From this list of dictionaries added after this, select the most recent company logo, according to the description. 
    Only return the link. {response['images']} """}]}
        ]
    logo_url = (get_response(prompt3).text).strip()
    # Retreive the image
    image = requests.get(logo_url)
    # Save to file
    if image.status_code == 200:
        file_extension = os.path.splitext(logo_url)[1]
        with open(rf"logos\{company_name}{file_extension}", "wb") as file:
            file.write(image.content)
    else:
        print(f"Failed to retrieve the image for {company_name}. Status code:", image.status_code)


get_logo("Proximus")

### Get date and time to include in prompt

In [17]:
from datetime import datetime

def get_datetime():
    current_datetime = datetime.now()
    day_of_week_str = current_datetime.strftime("%A")
    return f"The current date and time is {current_datetime}, today is a {day_of_week_str}."

get_datetime()

'The current date and time is 2025-01-17 13:22:05.390546, today is a Friday.'